In [1]:
import numpy as np
import os
from ICARUS.core.types import FloatArray
from ICARUS.vehicle.plane import Airplane
from ICARUS.vehicle.surface import WingSurface
from ICARUS.vehicle.utils import SymmetryAxes
from ICARUS.vehicle.plane import Airplane
from ICARUS.vehicle.wing_segment import WingSegment
from ICARUS.computation.solvers.AVL.files.input import avl_geo
from ICARUS.computation.solvers.AVL.analyses.pertrubations import avl_dynamic_analysis_fd
from ICARUS.computation.solvers.AVL.analyses.pertrubations import process_avl_fd_res
from ICARUS.environment.definition import EARTH_ISA
from ICARUS.flight_dynamics.state import State
from ICARUS.database import DB3D


In [2]:
from typing import Union
import copy

In [9]:
origin= np.array([0.0, 0.0, 0.0], dtype=float)

wing_position= np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)
wing_orientation=np.array(
    [0, 5.0, 0.0],
    dtype=float,
)
move_surf_w = {
    "names":["flap"],
    "hinges" : [0.7],
    "local_axes":[np.array([0,1.0,0.])],
    "span_percs" : [0.3],
    "rotation" : [1],
    "gains":[1.]

}
move_surf_el = {
    "names":["elev"],
    "hinges" : [0.5],
    "local_axes":[np.array([0,1.0,0.])],
    "span_percs" : [1],
    "rotation" : [1],
    "gains":[1.]

}

main_wing = WingSegment(
    name="wing",
    root_airfoil="4415",
    origin=origin + wing_position,
    orientation=wing_orientation,
    symmetries=SymmetryAxes.Y,
    span=2 * 1.130,
    sweep_offset=0,
    root_chord=0.159,
    tip_chord=0.072,
    twist_root=0,
    twist_tip=0,
    N=15,
    M=5,
    mass=0.670,
    moving_surfs= move_surf_w
)

elevator_pos: FloatArray = np.array(
    [0.54, 0.0, 0.0],
    dtype=float,
)
elevator_orientantion: FloatArray = np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)

elevator = WingSegment(
    name="elevator",
    root_airfoil="NACA0008",
    origin=origin + elevator_pos,
    orientation=elevator_orientantion,
    symmetries=SymmetryAxes.Y,
    span=2 * 0.169,
    sweep_offset=0,
    root_dihedral_angle=0,
    root_chord=0.130,
    tip_chord=0.03,
    N=15,
    M=5,
    mass=0.06,
    moving_surfs=move_surf_el
)

rudder_position: FloatArray = np.array(
    [0.47, 0.0, 0.01],
    dtype=float,
)
rudder_orientation: FloatArray = np.array(
    [0.0, 0.0, 90.0],
    dtype=float,
)

rudder = WingSegment(
    name="rudder",
    root_airfoil="NACA0008",
    origin=origin + rudder_position,
    orientation=rudder_orientation,
    symmetries=SymmetryAxes.NONE,
    span=0.160,
    sweep_offset=0.1,
    root_dihedral_angle=0,
    root_chord=0.2,
    tip_chord=0.1,
    N=15,
    M=5,
    mass=0.04,
)

lifting_surfaces: list[WingSurface] = [main_wing, elevator, rudder]

point_masses = [
    (0.500, np.array([-0.40, 0.0, 0.0], dtype=float), "engine"),  # Engine
    (1.000, np.array([0.090, 0.0, 0.0], dtype=float), "structure"),  # Structure
    # (1.000, np.array([0.090, 0.0, 0.0], dtype=float), "battery"),  # Battery
    (0.900, np.array([0.130, 0.0, 0.0], dtype=float), "payload"),  # Payload
]

In [10]:
plane = Airplane("flapped_wing",lifting_surfaces)
plane.add_point_masses(point_masses)


In [11]:
plane.directory

'flapped_wing'

In [12]:

UINF = 20
solver2D = "Xfoil"
angles = np.linspace(-10, 10, 11)

state = State(
        name="Unstick",
        airplane=plane,
        environment=EARTH_ISA,
        u_freestream= UINF
)

epsilons = {
    "u": 0.01,
    "w": 0.01,
    "q": 0.1,
    "theta": 0.01,
    "v": 0.01,
    "p": 0.1,
    "r": 0.1,
    "phi": 0.01,
}


In [13]:
state.add_all_pertrubations("Central", epsilons)
avl_dynamic_analysis_fd(plane, state, solver2D)
df = process_avl_fd_res(plane, state)
state.set_pertrubation_results(df)
state.stability_fd()

Trimming the plane
Searching for airfoil naca0008 in DB2D
Loading airfoil naca0008 from DB2D


TrimOutsidePolars: 